In [1]:
from pycaret.classification import *
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from datetime import date, timedelta
import time
import random
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize
from sklearn.cluster import KMeans
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [3]:
nn_path = 'C:/404/404/'

train_err = pd.read_csv(nn_path + 'train_err2.csv')
train_qui = pd.read_csv(nn_path +  'train_qui2.csv')
train_prob = pd.read_csv(nn_path +  'train_prob2.csv')

test_err = pd.read_csv(nn_path +  'test_err2.csv')
test_qui = pd.read_csv(nn_path +  'test_qui2.csv')

In [4]:
path = 'C:/404/404/preprocessed/mk2_with_categorized/'

test_err_42_counts = pd.read_csv(path + 'test_err_42_counts.csv', index_col=['Unnamed: 0'])
train_err_42_counts = pd.read_csv(path + 'train_err_42_counts.csv', index_col=['Unnamed: 0'])

test_err_sum_byday = pd.read_csv(path +'test_err_sum_byday.csv', index_col=['Unnamed: 0'])
train_err_sum_byday = pd.read_csv(path +'train_err_sum_byday.csv', index_col=['Unnamed: 0'])

test_qui_sum_byday = pd.read_csv(path +'test_qui_sum_byday.csv', index_col=['Unnamed: 0'])
train_qui_sum_byday = pd.read_csv(path +'train_qui_sum_byday.csv', index_col=['Unnamed: 0'])

test_2h = pd.read_csv(path +'test_2h.csv', index_col=['Unnamed: 0'])
train_2h = pd.read_csv(path +'train_2h.csv', index_col=['Unnamed: 0'])

train_6h = pd.read_csv(path + 'train_6h.csv', index_col=['Unnamed: 0'])
test_6h = pd.read_csv(path + 'test_6h.csv', index_col=['Unnamed: 0'])

train_12h = pd.read_csv(path + 'train_12h.csv', index_col=['Unnamed: 0'])
test_12h = pd.read_csv(path + 'test_12h.csv', index_col=['Unnamed: 0'])

## 퀼리티 데이터를 확률로 환산하여 평균한 값
train_qui_prob = pd.read_csv(path + 'train_qui_prob.csv')#, index_col=['Unnamed: 0'])
test_qui_prob = pd.read_csv(path + 'test_qui_prob.csv')#, index_col=['Unnamed: 0'])

train_qui_prob.set_index(keys = 'user_id', inplace=True)
test_qui_prob.set_index(keys = 'user_id', inplace=True)

train_errcode = pd.read_csv(path + 'train_errcode.csv')
test_errcode = pd.read_csv(path + 'test_errcode.csv')

train_errcode.set_index(keys = 'user_id', inplace = True)
test_errcode.set_index(keys = 'user_id', inplace = True)

In [5]:
### 43262 없따...

In [6]:
test_errcode

,Err_code__0,Err_code__1,Err_code__100,Err_code__13,Err_code__14,Err_code__2,Err_code__21,Err_code__3,Err_code__4,Err_code__5,Err_code__6,Err_code__78,Err_code__79,Err_code__8.0,Err_code__80,Err_code__81,Err_code__82,Err_code__83,Err_code__84,Err_code__85,Err_code__86,Err_code__87,Err_code__88,Err_code__89,Err_code__90,Err_code__91,Err_code__94,Err_code__95,Err_code__99,Err_code__B-51049,Err_code__B-A8002,Err_code__ETC,Err_code__H-51042,Err_code__NFANDROID2,Err_code__Q-64002,Err_code__S-61001,Err_code__S-65002,Err_code__UNKNOWN,Err_code__V-21008,Err_code__active,Err_code__connection fail for LMP response timout,Err_code__connection fail to establish,Err_code__connection timeout,Err_code__connectionterminated by local host,Err_code__standby,Err_code__terminate by peer user
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
30000,90.0,1251.0,0.0,0.0,14.0,15.0,0.0,5.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,0.0,0.0,0.0,0.0,9.0,0.0,1.0,0.0,141.0,0.0,55.0,1107.0,0.0,4.0,0.0
30001,98.0,180.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30002,186.0,614.0,0.0,0.0,4.0,13.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,7.0,0.0,0.0,26.0,0.0,6.0,1.0
30003,91.0,147.0,0.0,0.0,11.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,106.0,0.0,0.0,8.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
30004,193.0,395.0,0.0,1.0,25.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,74.0,1.0,74.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,90.0,0.0,6.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44994,139.0,624.0,0.0,5.0,26.0,14.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,87.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,7.0,197.0,0.0,0.0,0.0
44995,105.0,370.0,0.0,2.0,6.0,13.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44996,251.0,698.0,0.0,3.0,4.0,529.0,0.0,573.0,6.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,104.0,0.0,4.0,44.0,0.0,11.0,2.0


In [7]:
train_time_series_6h = pd.read_csv(path + 'train_time_series_6h.csv', index_col=['Unnamed: 0'])
test_time_series_6h = pd.read_csv(path + 'test_time_series_6h.csv', index_col=['Unnamed: 0'])
train_time_series_6h.drop(['Group', 'problems'], axis = 1, inplace = True)

prob_list = list(set(train_prob.user_id))

In [8]:
train_2h.rename(columns = lambda x: x + '_2', inplace = True)
test_2h.rename(columns = lambda x: x + '_2', inplace = True)

train_6h.rename(columns = lambda x: x + '_6', inplace = True)
test_6h.rename(columns = lambda x: x + '_6', inplace = True)

train_12h.rename(columns = lambda x: x + '_12', inplace = True)
test_12h.rename(columns = lambda x: x + '_12', inplace = True)

In [9]:
# 각 사용자가 42가지 에러타입을 몇 번씩 겪었는지 나타냄
train_err_42_counts

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
10000,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10001,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0
10002,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10003,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0
10004,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,0.0,5.0,10.0,8.0,8.0,26.0,18.0,0.0,0.0,0.0,0.0,0.0,17.0,12.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,31.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0
24996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24997,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,0.0,16.0,18.0,4.0,1.0,181.0,138.0,0.0,0.0,0.0,0.0,0.0,50.0,29.0,0.0,0.0,49.0,0.0,0.0,0.0,0.0,223.0,0.0,16.0,17.0,0.0,1.0,1.0,0.0,0.0,58.0,8.0,5.0
24998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,0.0,1.0,51.0,12.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,14.0,0.0,15.0,12.0,0.0,1.0,1.0,0.0,0.0,6.0,0.0,0.0


In [10]:
# 일단 자료들 날짜형식이나 콤마(,) 제거 등 간단한 전처리 진행

train_qui2 = train_qui.copy()
test_qui2 = test_qui.copy()
train_err2 = train_err.copy()
test_err2 = test_err.copy()

# 퀼리티 3,4는 값이 1밖에 없어서 의미가 없기 때문에 제외
# 펌웨어 버전은 이미 err 자료에 있기 때문에 중복으로 제외
# 퀼리티 0은 EDA 결과 퀼리티 2와 피어슨 상관계수 0.99를 보이기 때문에 0과 2 중에 하나만 골랐음
# 0의 결측치 Nan은 15%고 2의 결측치는 4%기 때문에 결측치가 적은 2를 남기고 0은 drop 시킨다.

## 퀼리티 6과 7도 0.81로 강한 상관관계를 보이고 있는데, 하나를 제외해야 하나 고민이 됨
train_qui2.drop(['fwver','quality_0', 'quality_3', 'quality_4'], axis =1, inplace=True)
test_qui2.drop(['fwver', 'quality_0', 'quality_3', 'quality_4'], axis =1, inplace=True)

test_qui2 = test_qui2.replace(',','', regex=True)
train_qui2 = train_qui2.replace(',','', regex=True)
test_qui2.loc[:,'quality_1':'quality_12'] = test_qui2.loc[:,'quality_1':'quality_12'].astype(float)
train_qui2.loc[:,'quality_1':'quality_12'] = train_qui2.loc[:,'quality_1':'quality_12'].astype(float)

train_qui2['time'] = pd.to_datetime(train_qui['time'], format = '%Y-%m-%d %H:%M:%S')
test_qui2['time'] = pd.to_datetime(test_qui['time'], format = '%Y-%m-%d %H:%M:%S')

train_err2['time'] = pd.to_datetime(train_err2['time'], format = '%Y-%m-%d %H:%M:%S')
test_err2['time'] = pd.to_datetime(test_err2['time'], format = '%Y-%m-%d %H:%M:%S')

train_err4 = train_err2.copy()
test_err4 = test_err2.copy()

In [11]:
##### 퀼리티 데이터, 하루에 2개 이상 수집되는 경우(에러 빈도가 높은 경우)

def quality_one_day(dataset):
  data = dataset.copy()
  ids = data['user_id'].unique()
  ids.sort()
  data['day'] = data['time'].dt.dayofyear
  df_list = []
  for id in tqdm(ids):
    temp = data[data['user_id'] == id]
    d_list = temp.day.unique()
    too_much = 0
    for d in d_list:
      temp2 = temp[temp['day'] == d]
      err = len(temp2)/12
      if err > 1:
        too_much += err
    df_list.append(too_much)

  df = pd.DataFrame(df_list, index = ids, columns = ['overlap'])
  return df

In [12]:
train_tm = quality_one_day(train_qui2)
test_tm = quality_one_day(test_qui2)

In [13]:
# 퀼리티 임계치
# [-1, 0, 1] == -1 카테고리 -1, -1~0 =카테고리 0, 그 이상 = 카테고리 1
qualities = {}

qualities['quality_1'] = [-1,0, 1]
qualities['quality_2'] = [-1,0, 1]
qualities['quality_5'] = [-1,0,1,2,4, 10]
qualities['quality_6'] = [-1,0, 1]
qualities['quality_7'] = [0,1,100, 101]
qualities['quality_8'] = [0,1,2]
qualities['quality_9'] = [0,1,2]
qualities['quality_10'] = [0,1,2,3, 4, 5,7, 10 ,14,15] ##??
qualities['quality_11'] = [-1,0,1]
qualities['quality_12'] = [0,1,2]

## 모델 임계치
## 모델 0,1,2,3,4의 비율을 더하면 99.999% 이므로, 모델 5~8은 other model로 통합하는 것이 바람직 해 보임.

In [14]:
def quality_categorize(dataset):
  data = dataset.copy()
  for key in tqdm(qualities):
    intervals = qualities.get(key)
    for i in range(len(intervals)):
      if intervals[i] == intervals[-1]:
        data[key][data[key] >= intervals[i]] =  intervals[i]
      # elif intervals[i+1] - intervals[i] == 1:
      #   continue
      else:
        data[key][(data[key] >= intervals[i]) & (data[key] < intervals[i+1])] = intervals[i]
  return data

###### 그냥 쓰자니 불일치가 너무 많아서... 카테고리화 해서 확률로 전환한다.

# train_qui3 = quality_categorize(train_qui2)
# test_qui3 = quality_categorize(test_qui2)

In [15]:
### 펌웨어의 확률을 구하는 함수 ###
## 유진's 코드를 참조하여 좀더 빠르게 구동되도록 수정함.

def fw_probs(traindata, testdata):
  
  train_temp = traindata.drop_duplicates(subset = 'user_id')
  test_temp = testdata.drop_duplicates(subset = 'user_id')
  

  test_list = []
  train_temp['problems'] = train_temp['user_id'].isin(prob_list)
  train_tr = train_temp[train_temp['problems']]
  train_fls = train_temp[train_temp['problems'] != True]

  new_fw = ['03.11', '04.16', '04.22', '04.33', '04.73', '04.82', '05.15', '05.66', '8.5', '10']

  for fw in tqdm(new_fw):
    if fw == '10':
      ori = len(train_temp[(train_temp['fwver'].str.contains(fw)) & ~(train_temp['fwver'].str.contains('104')) & ~(train_temp['fwver'].str.contains('109'))])
      tr = len(train_tr[(train_tr['fwver'].str.contains(fw)) & ~(train_tr['fwver'].str.contains('104')) & ~(train_tr['fwver'].str.contains('109'))])
      fls = len(train_fls[(train_temp['fwver'].str.contains(fw)) & ~(train_fls['fwver'].str.contains('104')) & ~(train_fls['fwver'].str.contains('109'))])
      fw_st = [ori, tr, fls, tr/ori]
      test_list.append(fw_st)
    else:
      ori = len(train_temp[train_temp['fwver'].str.contains(fw)])
      tr = len(train_tr[train_tr['fwver'].str.contains(fw)])
      fls = len(train_fls[train_fls['fwver'].str.contains(fw)])
      fw_st = [ori, tr, fls, tr/ori]
      test_list.append(fw_st)

  fw_df = pd.DataFrame(test_list, index = new_fw, columns = ['원본', '불만고객', '고객', '불만확률'])

  train_temp['fw_prob'] = 0
  test_temp['fw_prob'] = 0

  for fw in tqdm(new_fw):
    if fw == '10':
      train_temp['fw_prob'][(train_temp['fwver'].str.contains(fw)) & ~(train_temp['fwver'].str.contains('104')) & ~(train_temp['fwver'].str.contains('109'))] = fw_df.loc[fw][3]
      test_temp['fw_prob'][(test_temp['fwver'].str.contains(fw)) & ~(test_temp['fwver'].str.contains('104')) & ~(test_temp['fwver'].str.contains('109'))] = fw_df.loc[fw][3]
    else:
      train_temp['fw_prob'][train_temp['fwver'].str.contains(fw)] = fw_df.loc[fw][3]
      test_temp['fw_prob'][test_temp['fwver'].str.contains(fw)] = fw_df.loc[fw][3]
  df = train_temp.loc[:,['user_id', 'fw_prob']]
  df_t = test_temp.loc[:,['user_id', 'fw_prob']]

  df.set_index('user_id', inplace = True)
  df_t.set_index('user_id', inplace = True)

  return df, df_t

In [16]:
train_fw, test_fw = fw_probs(train_err2, test_err2)

In [17]:
# Error 데이터를 일일 단위로 쪼개서 각 사용자가 날짜별로 겪은 에러코드의 누계
# 0일은 10월 31일, 마지막은 11월 30일
train_err_sum_byday

,er_sum_0,er_sum_1,er_sum_2,er_sum_3,er_sum_4,er_sum_5,er_sum_6,er_sum_7,er_sum_8,er_sum_9,er_sum_10,er_sum_11,er_sum_12,er_sum_13,er_sum_14,er_sum_15,er_sum_16,er_sum_17,er_sum_18,er_sum_19,er_sum_20,er_sum_21,er_sum_22,er_sum_23,er_sum_24,er_sum_25,er_sum_26,er_sum_27,er_sum_28,er_sum_29,er_sum_30
10000,0,11,20,38,43,53,62,82,89,94,104,118,123,140,149,165,174,182,199,218,223,230,235,240,245,254,265,277,295,308,317
10001,0,11,61,90,138,180,209,222,246,299,351,374,413,421,462,486,501,531,556,583,628,653,671,689,716,727,785,2237,2275,2337,2365
10002,0,10,23,36,51,60,68,85,96,108,118,129,140,145,160,173,188,193,202,217,222,232,248,251,257,268,273,289,289,297,306
10003,0,9,23,33,38,54,59,61,72,77,96,120,126,132,134,136,151,157,164,179,184,191,212,223,240,246,254,275,286,300,306
10004,0,25,46,95,123,134,162,180,209,229,283,302,323,340,360,375,390,408,449,457,559,587,603,621,644,671,690,714,732,755,777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,41,58,76,76,100,103,133,155,172,194
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
24997,0,9,43,77,141,163,194,218,248,274,292,301,320,339,377,408,418,445,464,496,525,550,579,621,654,695,718,748,779,813,826
24998,0,7,7,17,17,24,24,32,32,39,76,83,83,89,91,96,102,102,111,112,118,120,125,131,131,137,137,144,144,151,155


In [18]:
# Quality 데이터를 일일 단위로 쪼개서 퀼리티 로그가 사용자 별로 몇 번 수집되었나
# 누적, 마지막 11월 30일 자료는 최종 몇 번의 로그 수집이 있었는가를 의미함.
train_qui_sum_byday

,q_sum_0,q_sum_1,q_sum_2,q_sum_3,q_sum_4,q_sum_5,q_sum_6,q_sum_7,q_sum_8,q_sum_9,q_sum_10,q_sum_11,q_sum_12,q_sum_13,q_sum_14,q_sum_15,q_sum_16,q_sum_17,q_sum_18,q_sum_19,q_sum_20,q_sum_21,q_sum_22,q_sum_23,q_sum_24,q_sum_25,q_sum_26,q_sum_27,q_sum_28,q_sum_29,q_sum_30
10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
10002,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,5.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,8.0
10004,0.0,0.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
10005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
10006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
24992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
24993,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
24995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0


In [19]:
# train_err_42_counts, train_err_code_frq, train_qui_frq, train_err_sum_byday, train_qui_sum_byday
# test_err3, test_err_42_counts, test_err_code_frq, test_qui_frq, test_err_sum_byday, test_qui_sum_byday
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
robust = RobustScaler()
robust_q = RobustScaler()
robust_nq = RobustScaler()
minmax = MinMaxScaler()

In [20]:
# 24시간을 단순하게 평균내면, 23시와 1시의 평균이 이상해지기 때문에(실제로는 2시간 차이인데, 산술적으로 22시간 차이로 잡힘)
# 태양광 때 처럼 일단 코사인 시간으로 바꿔서 평균, 분산, 최빈값을 구한다.

def cos_time(dataset):
  dataset['cos_time'] = np.cos(2*np.pi*dataset.hour/24)
  dataset['sin_time'] = np.sin(2*np.pi*dataset.hour/24)
  return dataset

def cos_stats(dataset):
  dataset['hour'] = dataset['time'].dt.hour
  data = cos_time(dataset)[['user_id', 'cos_time', 'sin_time']]
  cols = []
  temp = data.groupby(by = 'user_id')

  a = temp['cos_time'].mean()
  cols += ['cos_mean']
  b = temp['cos_time'].std()
  cols += ['cos_std']

  c = temp['sin_time'].mean()
  cols += ['sin_mean']
  d = temp['sin_time'].std()
  cols += ['sin_std']

  df = pd.concat([a,b,c,d], axis = 1) 
  df.columns = cols
  return df

In [21]:
cos_train = cos_stats(train_err4)
cos_test = cos_stats(test_err4)

In [22]:
cos_test

,cos_mean,cos_std,sin_mean,sin_std
user_id,,,,
30000,0.087315,0.587431,-0.482636,0.643926
30001,-0.008207,0.684306,0.059167,0.729163
30002,-0.088347,0.631359,-0.002706,0.771122
30003,0.079423,0.656741,-0.008234,0.751665
30004,-0.003992,0.687212,0.059433,0.724792
...,...,...,...,...
44994,-0.102470,0.684066,-0.355426,0.629286
44995,0.042984,0.769687,-0.180729,0.612331
44996,-0.000892,0.653807,-0.260090,0.710849


In [23]:
# errtype 1~42까지 확률로 치환하기.

def errtype_to_prob(trainset, testset):
  data = trainset.copy()
  t_data = testset.copy()
  data['problems'] = data['user_id'].isin(prob_list)
  one_hot = pd.get_dummies(data['errtype'])

  temp = data['problems']
  temp2 = pd.concat([temp, one_hot], axis = 1)

  temp2_tr = temp2[temp2['problems']].fillna(0)
  temp2_fls = temp2[temp2['problems'] != True].fillna(0)

  trues = pd.DataFrame(temp2_tr.sum()).transpose()
  falses = pd.DataFrame(temp2_fls.sum()).transpose()
  tru = trues.drop('problems', axis = 1)
  fls = falses.drop('problems', axis = 1)
  rslt = tru/(tru+fls)
  er_list = list(set(data.errtype))
  er_list.sort()
  for er in tqdm(er_list):
    data.errtype[data['errtype'] == er] =rslt.loc[0,er]
    t_data.errtype[t_data['errtype'] == er] =rslt.loc[0,er]
  return data, t_data

In [24]:
train, test = errtype_to_prob(train_err2, test_err2)

In [25]:
train_errtype_mean = train.groupby(by = 'user_id').mean()[['errtype']]
train_errtype_std = train.groupby(by = 'user_id').std()[['errtype']]
train_errtype_prob = pd.concat([train_errtype_mean, train_errtype_std], axis = 1)
train_errtype_prob.columns = ['errtype_mean', 'errtype_std']


test_errtype_mean = test.groupby(by = 'user_id').mean()[['errtype']]
test_errtype_std = test.groupby(by = 'user_id').std()[['errtype']]
test_errtype_prob = pd.concat([test_errtype_mean, test_errtype_std], axis = 1)
test_errtype_prob.columns = ['errtype_mean', 'errtype_std']

In [26]:
train_errtype_prob

,errtype_mean,errtype_std
user_id,,
10000,0.376132,0.087355
10001,0.499675,0.062462
10002,0.356911,0.076342
10003,0.430163,0.051116
10004,0.456258,0.061915
...,...,...
24995,0.454295,0.079495
24996,0.475615,0.149939
24997,0.436103,0.063117


In [27]:
# 중간에 펌웨어 업데이트를 한 경우에 1, 아니면 0으로 표시하는 함수 

def update(dataset):

  data = dataset.copy()
  ids = list(set(data.user_id))
  ids.sort()
  print('1/3')
  data.replace({'fwver': np.nan}, {'fwver': '00.00.0000'}, inplace = True)
  data.replace({'fwver': '8.5.3'}, {'fwver': '08.05.3000'}, inplace = True)
  data.replace({'fwver': '10'}, {'fwver': '10.00.0000'}, inplace = True)
  data['fwver'] = data['fwver'].str.replace('\.','', regex = True)
  print('2/3')
  data = data.astype({'fwver': 'int'})
  fw_list = data.fwver.unique()
  data2 = data.groupby(by = 'user_id').mean()
  print('3/3')
  data2['fw_update'] = data2['fwver'].isin(fw_list)
  data2['fw_update'] = data2['fw_update'].astype(int)
  df = data2.loc[:,['fw_update']]
  return df

In [28]:
fw_update_train = update(train_err2)
fw_update_test = update(test_err2)

1/3
2/3
3/3
1/3
2/3
3/3


In [29]:
train_err3 = train_err2[['user_id', 'errtype', 'errcode', 'time']]
train_err3['prob'] = train_err3['user_id'].isin(prob_list)
ec = pd.DataFrame(train_err3.errcode.value_counts())
err_code = list(ec[ec['errcode'] > 900].index)

In [30]:
train_err3 = train_err2[['user_id', 'errtype', 'errcode', 'time']]
test_err3 = test_err2[['user_id', 'errtype', 'errcode', 'time']]

# 상위 15개 리스트
# err_code =['1','0','connection timeout','B-A8002','80','79','14','active','2','84','85','standby','NFANDROID2','connection fail to establish','3']

train_err3['prob'] = train_err3['user_id'].isin(prob_list)
ec = pd.DataFrame(train_err3.errcode.value_counts())
err_code = list(ec[ec['errcode'] > 1000].index)

# 위의 15개에 포함되지 않는 에러코드는 걍 'etc'로 분류
train_err3['errcode'][~train_err3['errcode'].isin(err_code)] = 'ETC'
test_err3['errcode'][~test_err3['errcode'].isin(err_code)] = 'ETC'

In [31]:
dummi = pd.get_dummies(train_err3['errcode'], prefix = 'Err_code_')
id = train_err3[['user_id']]
train_types = pd.concat([id, dummi], axis =1)

dummi = pd.get_dummies(test_err3['errcode'], prefix = 'Err_code_')
id = test_err3[['user_id']]
test_types = pd.concat([id, dummi], axis =1)

In [32]:
k = train_types[train_types['user_id'] == 10000]
a = k.groupby(by = 'user_id').sum()

k = train_types[train_types['user_id'] == 10001]
b = k.groupby(by = 'user_id').sum()

In [33]:
df = pd.DataFrame(columns = train_types.columns)

In [34]:
df = pd.concat([df,b], axis = 0)
df

,user_id,Err_code__0,Err_code__1,Err_code__100,Err_code__13,Err_code__14,Err_code__2,Err_code__3,Err_code__4,Err_code__5,Err_code__6,Err_code__78,Err_code__79,Err_code__8.0,Err_code__80,Err_code__81,Err_code__82,Err_code__83,Err_code__84,Err_code__85,Err_code__86,Err_code__87,Err_code__88,Err_code__89,Err_code__90,Err_code__91,Err_code__94,Err_code__95,Err_code__99,Err_code__B-A8002,Err_code__ETC,Err_code__H-51042,Err_code__NFANDROID2,Err_code__Q-64002,Err_code__S-61001,Err_code__S-65002,Err_code__UNKNOWN,Err_code__V-21008,Err_code__active,Err_code__connection fail to establish,Err_code__connection timeout,Err_code__connectionterminated by local host,Err_code__standby,Err_code__terminate by peer user
10001,NaN,183.0,1274.0,0.0,1.0,16.0,7.0,5.0,18.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,0.0,0.0,56.0,0.0,0.0,0.0,0.0,0.0,126.0,0.0,0.0,0.0,625.0,0.0


In [35]:
df = pd.concat([df,a], axis = 0)
df

,user_id,Err_code__0,Err_code__1,Err_code__100,Err_code__13,Err_code__14,Err_code__2,Err_code__3,Err_code__4,Err_code__5,Err_code__6,Err_code__78,Err_code__79,Err_code__8.0,Err_code__80,Err_code__81,Err_code__82,Err_code__83,Err_code__84,Err_code__85,Err_code__86,Err_code__87,Err_code__88,Err_code__89,Err_code__90,Err_code__91,Err_code__94,Err_code__95,Err_code__99,Err_code__B-A8002,Err_code__ETC,Err_code__H-51042,Err_code__NFANDROID2,Err_code__Q-64002,Err_code__S-61001,Err_code__S-65002,Err_code__UNKNOWN,Err_code__V-21008,Err_code__active,Err_code__connection fail to establish,Err_code__connection timeout,Err_code__connectionterminated by local host,Err_code__standby,Err_code__terminate by peer user
10001,NaN,183.0,1274.0,0.0,1.0,16.0,7.0,5.0,18.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,0.0,0.0,56.0,0.0,0.0,0.0,0.0,0.0,126.0,0.0,0.0,0.0,625.0,0.0
10000,NaN,104.0,212.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
# 시계열 자료를 활용해 군집화.
ks = KMeans(n_clusters = 7, random_state = 42)
ks.fit(train_time_series_6h)
train_time_series_6h['Group'] = ks.predict(train_time_series_6h)
test_time_series_6h['Group'] = ks.predict(test_time_series_6h)
# train_time_series_6h['problems'] = train_time_series_6h.index.isin(prob_list)

In [37]:
train_time_series_6h

,2020-10-31 18:00:00,2020-11-01 00:00:00,2020-11-01 06:00:00,2020-11-01 12:00:00,2020-11-01 18:00:00,2020-11-02 00:00:00,2020-11-02 06:00:00,2020-11-02 12:00:00,2020-11-02 18:00:00,2020-11-03 00:00:00,2020-11-03 06:00:00,2020-11-03 12:00:00,2020-11-03 18:00:00,2020-11-04 00:00:00,2020-11-04 06:00:00,2020-11-04 12:00:00,2020-11-04 18:00:00,2020-11-05 00:00:00,2020-11-05 06:00:00,2020-11-05 12:00:00,2020-11-05 18:00:00,2020-11-06 00:00:00,2020-11-06 06:00:00,2020-11-06 12:00:00,2020-11-06 18:00:00,2020-11-07 00:00:00,2020-11-07 06:00:00,2020-11-07 12:00:00,2020-11-07 18:00:00,2020-11-08 00:00:00,2020-11-08 06:00:00,2020-11-08 12:00:00,2020-11-08 18:00:00,2020-11-09 00:00:00,2020-11-09 06:00:00,2020-11-09 12:00:00,2020-11-09 18:00:00,2020-11-10 00:00:00,2020-11-10 06:00:00,2020-11-10 12:00:00,2020-11-10 18:00:00,2020-11-11 00:00:00,2020-11-11 06:00:00,2020-11-11 12:00:00,2020-11-11 18:00:00,2020-11-12 00:00:00,2020-11-12 06:00:00,2020-11-12 12:00:00,2020-11-12 18:00:00,2020-11-13 00:00:00,2020-11-13 06:00:00,2020-11-13 12:00:00,2020-11-13 18:00:00,2020-11-14 00:00:00,2020-11-14 06:00:00,2020-11-14 12:00:00,2020-11-14 18:00:00,2020-11-15 00:00:00,2020-11-15 06:00:00,2020-11-15 12:00:00,2020-11-15 18:00:00,2020-11-16 00:00:00,2020-11-16 06:00:00,2020-11-16 12:00:00,2020-11-16 18:00:00,2020-11-17 00:00:00,2020-11-17 06:00:00,2020-11-17 12:00:00,2020-11-17 18:00:00,2020-11-18 00:00:00,2020-11-18 06:00:00,2020-11-18 12:00:00,2020-11-18 18:00:00,2020-11-19 00:00:00,2020-11-19 06:00:00,2020-11-19 12:00:00,2020-11-19 18:00:00,2020-11-20 00:00:00,2020-11-20 06:00:00,2020-11-20 12:00:00,2020-11-20 18:00:00,2020-11-21 00:00:00,2020-11-21 06:00:00,2020-11-21 12:00:00,2020-11-21 18:00:00,2020-11-22 00:00:00,2020-11-22 06:00:00,2020-11-22 12:00:00,2020-11-22 18:00:00,2020-11-23 00:00:00,2020-11-23 06:00:00,2020-11-23 12:00:00,2020-11-23 18:00:00,2020-11-24 00:00:00,2020-11-24 06:00:00,2020-11-24 12:00:00,2020-11-24 18:00:00,2020-11-25 00:00:00,2020-11-25 06:00:00,2020-11-25 12:00:00,2020-11-25 18:00:00,2020-11-26 00:00:00,2020-11-26 06:00:00,2020-11-26 12:00:00,2020-11-26 18:00:00,2020-11-27 00:00:00,2020-11-27 06:00:00,2020-11-27 12:00:00,2020-11-27 18:00:00,2020-11-28 00:00:00,2020-11-28 06:00:00,2020-11-28 12:00:00,2020-11-28 18:00:00,2020-11-29 00:00:00,2020-11-29 06:00:00,2020-11-29 12:00:00,2020-11-29 18:00:00,2020-11-30 00:00:00,2020-11-30 06:00:00,2020-11-30 12:00:00,2020-11-30 18:00:00,2020-12-01 00:00:00,2020-12-01 06:00:00,2020-12-01 12:00:00,2020-12-01 18:00:00,2020-12-02 00:00:00,2020-12-02 06:00:00,2020-12-02 12:00:00,2020-12-02 18:00:00,Group
10000,-0.63093,0.315465,-0.315465,0.000000,-0.630930,0.184535,-0.315465,-0.630930,0.000000,0.315465,-0.315465,0.460399,-0.630930,-0.630930,-0.630930,0.101557,-0.315465,0.254692,-0.315465,-0.630930,0.000000,0.369070,-0.315465,-0.630930,-0.630930,-0.130930,0.460399,0.101557,0.101557,0.101557,0.000000,-0.630930,-0.630930,-0.130930,0.000000,-0.630930,-0.630930,0.417022,-0.315465,-0.630930,-0.630930,0.500000,0.000000,-0.630930,-0.630930,-0.315465,-0.315465,-0.630930,0.000000,0.417022,0.000000,0.184535,-0.630930,-0.130930,0.000000,0.101557,-0.630930,0.536429,0.101557,-0.630930,-0.630930,0.184535,-0.315465,-0.630930,0.000000,0.315465,-0.315465,-0.630930,-0.630930,0.184535,0.000000,0.417022,-0.630930,-0.315465,0.101557,0.254692,0.369070,-0.315465,0.101557,-0.630930,-0.630930,0.000000,0.101557,-0.630930,-0.630930,-0.315465,0.101557,-0.630930,-0.630930,-0.315465,0.101557,-0.630930,-0.630930,-0.315465,0.101557,-0.630930,-0.630930,0.184535,-0.315465,-0.630930,0.000000,0.315465,-0.315465,-0.630930,0.000000,0.315465,-0.315465,-0.630930,0.101557,0.369070,-0.315465,-0.630930,0.417022,0.315465,0.254692,-0.630930,-0.630930,0.101557,-0.315465,-0.630930,0.101557,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,0
10001,-0.63093,0.369070,-0.630930,-0.630930,0.000000,0.709142,0.536429,0.536429,0.369070,-0.315465,0.630930,0.315465,0.254692,-0.130930,0.901593,-0.630930,0.709142,0.570157,-0.63

In [38]:
def multi_model(dataset):
  data = dataset.copy()
  #ids = [x for x in range(10000,12000)]
  ids = data.user_id.unique()
  ids.sort()

  models = []

  for id in tqdm(ids):
    temp = data[data['user_id'] == id]
    mul = len(temp.model_nm.unique())
    if mul == 1:
      models.append('one')
    else:
      models.append('multi')
    
  df = pd.DataFrame(models, index = ids, columns = ['multi_model'])

  return df

# tt = train_err2[train_err2.user_id == 10001]
# len(tt.fwver.unique())

In [39]:
train_multi_model = multi_model(train_err2)
test_multi_model = multi_model(test_err2)

In [40]:
n_path = 'C:/404/404/preprocessed/mk3_dacon/'

a = pd.read_csv(n_path +'a.csv') #, index_col=['Unnamed: 0'])
a.index = a.index + 10000
i = pd.read_csv(n_path +'i.csv') #, index_col=['Unnamed: 0'])
i.index = i.index + 10000
j = pd.read_csv(n_path +'j.csv') #, index_col=['Unnamed: 0'])
j.index = j.index + 10000
l = pd.read_csv(n_path +'l.csv') #, index_col=['Unnamed: 0'])
l.index = l.index + 10000
o = pd.read_csv(n_path +'o.csv') #, index_col=['Unnamed: 0'])
o.index = o.index + 10000
k = pd.read_csv(n_path +'k.csv') #, index_col=['Unnamed: 0'])
k.index = k.index + 10000
g = pd.read_csv(n_path +'g.csv') #, index_col=['Unnamed: 0'])
g.index = g.index + 10000
h = pd.read_csv(n_path +'h.csv') #, index_col=['Unnamed: 0'])
h.index = h.index + 10000

train_bonus = pd.concat([a,g,h,i,j,k,l,o], axis = 1)

test_path = 'C:/404/404/preprocessed/mk3_dacon/test/'

a = pd.read_csv(test_path +'a.csv') #, index_col=['Unnamed: 0'])
a.index = a.index + 30000
i = pd.read_csv(test_path +'i.csv') #, index_col=['Unnamed: 0'])
i.index = i.index + 30000
j = pd.read_csv(test_path +'j.csv') #, index_col=['Unnamed: 0'])
j.index = j.index + 30000
l = pd.read_csv(test_path +'l.csv') #, index_col=['Unnamed: 0'])
l.index = l.index + 30000
o = pd.read_csv(test_path +'o.csv') #, index_col=['Unnamed: 0'])
o.index = o.index + 30000
k = pd.read_csv(test_path +'k.csv') #, index_col=['Unnamed: 0'])
k.index = k.index + 30000
g = pd.read_csv(test_path +'g.csv') #, index_col=['Unnamed: 0'])
g.index = g.index + 30000
h = pd.read_csv(test_path +'h.csv') #, index_col=['Unnamed: 0'])
h.index = h.index + 30000

test_bonus = pd.concat([a,g,h,i,j,k,l,o], axis = 1)

In [41]:
train_qui0 = pd.read_csv(n_path +'트레인_퀼리티_0말고.csv' , index_col=['Unnamed: 0'])
test_qui0 = pd.read_csv(n_path +'테스트_퀼리티_0말고.csv' , index_col=['Unnamed: 0'])

train_qui_std = pd.read_csv(n_path +'트레인_퀼리티_분산.csv' , index_col=['user_id'])
test_qui_std = pd.read_csv(n_path +'테스트_퀼리티_분산.csv' , index_col=['user_id'])

train_qui_day = pd.read_csv(n_path +'트레인_퀼리티_일일.csv'  , index_col=['Unnamed: 0'])
test_qui_day = pd.read_csv(n_path +'테스트_퀼리티_일일.csv'  , index_col=['Unnamed: 0'])

train_qui_week = pd.read_csv(n_path +'트레인_퀼리티_주간.csv'  , index_col=['Unnamed: 0'])
test_qui_week = pd.read_csv(n_path +'테스트_퀼리티_주간.csv'  , index_col=['Unnamed: 0'])

train_qui_mean = pd.read_csv(n_path +'트레인_퀼리티_평균.csv'  , index_col=['user_id'])
test_qui_mean = pd.read_csv(n_path +'테스트_퀼리티_평균.csv'  , index_col=['user_id'])

train_qui_sum = pd.read_csv(n_path +'트레인_퀼리티_합.csv'  , index_col=['user_id'])
test_qui_sum = pd.read_csv(n_path +'테스트_퀼리티_합.csv'  , index_col=['user_id'])

test_qui_sum.drop(['quality_1_nan'], axis = 1, inplace = True)

In [42]:
trainset = pd.concat([train_qui0, train_qui_week, train_qui_sum, train_fw, train_bonus, train_err_42_counts, train_err_sum_byday, train_qui_prob, cos_train, train_errtype_prob, fw_update_train, train_errcode,train_tm , train_2h], axis = 1)
testset = pd.concat([test_qui0, test_qui_week, test_qui_sum, test_fw, test_bonus, test_err_42_counts, test_err_sum_byday, test_qui_prob, cos_test, test_errtype_prob, fw_update_test, test_errcode, test_tm, test_2h], axis = 1)


trainset2 = pd.DataFrame(robust.fit_transform(trainset), index = trainset.index, columns=trainset.columns)
testset2 = pd.DataFrame(robust.transform(testset), index = testset.index, columns=testset.columns)
trainset2.fillna(-3, inplace=True)
testset2.fillna(-3, inplace=True)


col = [str(x) + '_t' for x in range(len(trainset2.columns))]
trainset2.columns = col
testset2.columns = col

trainset3 = pd.concat([trainset2, train_time_series_6h, train_multi_model], axis = 1)
testset3 = pd.concat([testset2, test_time_series_6h, test_multi_model], axis = 1)

du = pd.get_dummies(trainset3['Group'], prefix = 'Group')
du2 = pd.get_dummies(trainset3['multi_model'], prefix = 'model')
trainset3 = pd.concat([trainset3, du, du2], axis = 1)
trainset3.drop(['Group', 'multi_model'], axis = 1, inplace=True)
du = pd.get_dummies(testset3['Group'], prefix = 'Group')
du2 = pd.get_dummies(testset3['multi_model'], prefix = 'model')
testset3 = pd.concat([testset3, du, du2], axis = 1)
testset3.drop(['Group', 'multi_model'], axis = 1, inplace=True)



trainset3.reset_index(inplace=True)
probs = list(set(train_prob.user_id))
trainset3['TARGET'] = trainset3['index'].apply(lambda x: 1 if x in probs else 0)
trainset3.set_index('index', inplace = True)

num = list(trainset3.columns)
num.remove('TARGET')

In [43]:
testset3

,0_t,1_t,2_t,3_t,4_t,5_t,6_t,7_t,8_t,9_t,10_t,11_t,12_t,13_t,14_t,15_t,16_t,17_t,18_t,19_t,20_t,21_t,22_t,23_t,24_t,25_t,26_t,27_t,28_t,29_t,30_t,31_t,32_t,33_t,34_t,35_t,36_t,37_t,38_t,39_t,40_t,41_t,42_t,43_t,44_t,45_t,46_t,47_t,48_t,49_t,50_t,51_t,52_t,53_t,54_t,55_t,56_t,57_t,58_t,59_t,60_t,61_t,62_t,63_t,64_t,65_t,66_t,67_t,68_t,69_t,70_t,71_t,72_t,73_t,74_t,75_t,76_t,77_t,78_t,79_t,80_t,81_t,82_t,83_t,84_t,85_t,86_t,87_t,88_t,89_t,90_t,91_t,92_t,93_t,94_t,95_t,96_t,97_t,98_t,99_t,100_t,101_t,102_t,103_t,104_t,105_t,106_t,107_t,108_t,109_t,110_t,111_t,112_t,113_t,114_t,115_t,116_t,117_t,118_t,119_t,120_t,121_t,122_t,123_t,124_t,125_t,126_t,127_t,128_t,129_t,130_t,131_t,132_t,133_t,134_t,135_t,136_t,137_t,138_t,139_t,140_t,141_t,142_t,143_t,144_t,145_t,146_t,147_t,148_t,149_t,150_t,151_t,152_t,153_t,154_t,155_t,156_t,157_t,158_t,159_t,160_t,161_t,162_t,163_t,164_t,165_t,166_t,167_t,168_t,169_t,170_t,171_t,172_t,173_t,174_t,175_t,176_t,177_t,178_t,179_t,180_t,181_t,182_t,183_t,184_t,185_t,186_t,187_t,188_t,189_t,190_t,191_t,192_t,193_t,194_t,195_t,196_t,197_t,198_t,199_t,200_t,201_t,202_t,203_t,204_t,205_t,206_t,207_t,208_t,209_t,210_t,211_t,212_t,213_t,214_t,215_t,216_t,217_t,218_t,219_t,220_t,221_t,222_t,223_t,224_t,225_t,226_t,227_t,228_t,229_t,230_t,231_t,232_t,233_t,234_t,235_t,236_t,237_t,238_t,239_t,240_t,241_t,242_t,243_t,244_t,245_t,246_t,247_t,248_t,249_t,...,511_t,512_t,513_t,514_t,515_t,516_t,517_t,518_t,519_t,520_t,521_t,522_t,523_t,524_t,525_t,526_t,527_t,528_t,529_t,530_t,531_t,532_t,533_t,534_t,535_t,536_t,537_t,538_t,539_t,540_t,541_t,542_t,543_t,544_t,545_t,546_t,547_t,548_t,549_t,550_t,551_t,552_t,553_t,554_t,555_t,556_t,557_t,558_t,559_t,560_t,561_t,562_t,563_t,564_t,565_t,566_t,567_t,568_t,569_t,570_t,571_t,572_t,573_t,574_t,575_t,576_t,577_t,578_t,579_t,580_t,581_t,582_t,583_t,584_t,585_t,586_t,587_t,588_t,589_t,590_t,591_t,592_t,593_t,594_t,595_t,596_t,597_t,598_t,599_t,600_t,601_t,602_t,603_t,604_t,605_t,606_t,607_t,608_t,609_t,610_t,611_t,612_t,613_t,614_t,615_t,616_t,617_t,618_t,619_t,620_t,621_t,622_t,2020-10-31 18:00:00,2020-11-01 00:00:00,2020-11-01 06:00:00,2020-11-01 12:00:00,2020-11-01 18:00:00,2020-11-02 00:00:00,2020-11-02 06:00:00,2020-11-02 12:00:00,2020-11-02 18:00:00,2020-11-03 00:00:00,2020-11-03 06:00:00,2020-11-03 12:00:00,2020-11-03 18:00:00,2020-11-04 00:00:00,2020-11-04 06:00:00,2020-11-04 12:00:00,2020-11-04 18:00:00,2020-11-05 00:00:00,2020-11-05 06:00:00,2020-11-05 12:00:00,2020-11-05 18:00:00,2020-11-06 00:00:00,2020-11-06 06:00:00,2020-11-06 12:00:00,2020-11-06 18:00:00,2020-11-07 00:00:00,2020-11-07 06:00:00,2020-11-07 12:00:00,2020-11-07 18:00:00,2020-11-08 00:00:00,2020-11-08 06:00:00,2020-11-08 12:00:00,2020-11-08 18:00:00,2020-11-09 00:00:00,2020-11-09 06:00:00,2020-11-09 12:00:00,2020-11-09 18:00:00,2020-11-10 00:00:00,2020-11-10 06:00:00,2020-11-10 12:00:00,2020-11-10 18:00:00,2020-11-11 00:00:00,2020-11-11 06:00:00,2020-11-11 12:00:00,2020-11-11 18:00:00,2020-11-12 00:00:00,2020-11-12 06:00:00,2020-11-12 12:00:00,2020-11-12 18:00:00,2020-11-13 00:00:00,2020-11-13 06:00:00,2020-11-13 12:00:00,2020-11-13 18:00:00,2020-11-14 00:00:00,2020-11-14 06:00:00,2020-11-14 12:00:00,2020-11-14 18:00:00,2020-11-15 00:00:00,2020-11-15 06:00:00,2020-11-15 12:00:00,2020-11-15 18:00:00,2020-11-16 00:00:00,2020-11-16 06:00:00,2020-11-16 12:00:00,2020-11-16 18:00:00,2020-11-17 00:00:00,2020-11-17 06:00:00,2020-11-17 12:00:00,2020-11-17 18:00:00,2020-11-18 00:00:00,2020-11-18 06:00:00,2020-11-18 12:00:00,2020-11-18 18:00:00,2020-11-19 00:00:00,2020-11-19 06:00:00,2020-11-19 12:00:00,2020-11-19 18:00:00,2020-11-20 00:00:00,2020-11-20 06:00:00,2020-11-20 12:00:00,2020-11-20 18:00:00,2020-11-21 00:00:00,2020-11-21 06:00:00,2020-11-21 12:00:00,2020-11-21 18:00:00,2020-11-22 00:00:00,2020-11-22 06:00:00,2020-11-22 12:00:00,2020-11-22 18:00:00,2020-11-23 00:00:00,2020-11-23 06:00:00,2020-11-23 12:00:00,2020-11-23 18:00:00,2020-11-24 00:00:00,2020-11-24 06:00:00,2020-11-24 12:00:00,2020-11-24 18

In [44]:
trainset3

,0_t,1_t,2_t,3_t,4_t,5_t,6_t,7_t,8_t,9_t,10_t,11_t,12_t,13_t,14_t,15_t,16_t,17_t,18_t,19_t,20_t,21_t,22_t,23_t,24_t,25_t,26_t,27_t,28_t,29_t,30_t,31_t,32_t,33_t,34_t,35_t,36_t,37_t,38_t,39_t,40_t,41_t,42_t,43_t,44_t,45_t,46_t,47_t,48_t,49_t,50_t,51_t,52_t,53_t,54_t,55_t,56_t,57_t,58_t,59_t,60_t,61_t,62_t,63_t,64_t,65_t,66_t,67_t,68_t,69_t,70_t,71_t,72_t,73_t,74_t,75_t,76_t,77_t,78_t,79_t,80_t,81_t,82_t,83_t,84_t,85_t,86_t,87_t,88_t,89_t,90_t,91_t,92_t,93_t,94_t,95_t,96_t,97_t,98_t,99_t,100_t,101_t,102_t,103_t,104_t,105_t,106_t,107_t,108_t,109_t,110_t,111_t,112_t,113_t,114_t,115_t,116_t,117_t,118_t,119_t,120_t,121_t,122_t,123_t,124_t,125_t,126_t,127_t,128_t,129_t,130_t,131_t,132_t,133_t,134_t,135_t,136_t,137_t,138_t,139_t,140_t,141_t,142_t,143_t,144_t,145_t,146_t,147_t,148_t,149_t,150_t,151_t,152_t,153_t,154_t,155_t,156_t,157_t,158_t,159_t,160_t,161_t,162_t,163_t,164_t,165_t,166_t,167_t,168_t,169_t,170_t,171_t,172_t,173_t,174_t,175_t,176_t,177_t,178_t,179_t,180_t,181_t,182_t,183_t,184_t,185_t,186_t,187_t,188_t,189_t,190_t,191_t,192_t,193_t,194_t,195_t,196_t,197_t,198_t,199_t,200_t,201_t,202_t,203_t,204_t,205_t,206_t,207_t,208_t,209_t,210_t,211_t,212_t,213_t,214_t,215_t,216_t,217_t,218_t,219_t,220_t,221_t,222_t,223_t,224_t,225_t,226_t,227_t,228_t,229_t,230_t,231_t,232_t,233_t,234_t,235_t,236_t,237_t,238_t,239_t,240_t,241_t,242_t,243_t,244_t,245_t,246_t,247_t,248_t,249_t,...,512_t,513_t,514_t,515_t,516_t,517_t,518_t,519_t,520_t,521_t,522_t,523_t,524_t,525_t,526_t,527_t,528_t,529_t,530_t,531_t,532_t,533_t,534_t,535_t,536_t,537_t,538_t,539_t,540_t,541_t,542_t,543_t,544_t,545_t,546_t,547_t,548_t,549_t,550_t,551_t,552_t,553_t,554_t,555_t,556_t,557_t,558_t,559_t,560_t,561_t,562_t,563_t,564_t,565_t,566_t,567_t,568_t,569_t,570_t,571_t,572_t,573_t,574_t,575_t,576_t,577_t,578_t,579_t,580_t,581_t,582_t,583_t,584_t,585_t,586_t,587_t,588_t,589_t,590_t,591_t,592_t,593_t,594_t,595_t,596_t,597_t,598_t,599_t,600_t,601_t,602_t,603_t,604_t,605_t,606_t,607_t,608_t,609_t,610_t,611_t,612_t,613_t,614_t,615_t,616_t,617_t,618_t,619_t,620_t,621_t,622_t,2020-10-31 18:00:00,2020-11-01 00:00:00,2020-11-01 06:00:00,2020-11-01 12:00:00,2020-11-01 18:00:00,2020-11-02 00:00:00,2020-11-02 06:00:00,2020-11-02 12:00:00,2020-11-02 18:00:00,2020-11-03 00:00:00,2020-11-03 06:00:00,2020-11-03 12:00:00,2020-11-03 18:00:00,2020-11-04 00:00:00,2020-11-04 06:00:00,2020-11-04 12:00:00,2020-11-04 18:00:00,2020-11-05 00:00:00,2020-11-05 06:00:00,2020-11-05 12:00:00,2020-11-05 18:00:00,2020-11-06 00:00:00,2020-11-06 06:00:00,2020-11-06 12:00:00,2020-11-06 18:00:00,2020-11-07 00:00:00,2020-11-07 06:00:00,2020-11-07 12:00:00,2020-11-07 18:00:00,2020-11-08 00:00:00,2020-11-08 06:00:00,2020-11-08 12:00:00,2020-11-08 18:00:00,2020-11-09 00:00:00,2020-11-09 06:00:00,2020-11-09 12:00:00,2020-11-09 18:00:00,2020-11-10 00:00:00,2020-11-10 06:00:00,2020-11-10 12:00:00,2020-11-10 18:00:00,2020-11-11 00:00:00,2020-11-11 06:00:00,2020-11-11 12:00:00,2020-11-11 18:00:00,2020-11-12 00:00:00,2020-11-12 06:00:00,2020-11-12 12:00:00,2020-11-12 18:00:00,2020-11-13 00:00:00,2020-11-13 06:00:00,2020-11-13 12:00:00,2020-11-13 18:00:00,2020-11-14 00:00:00,2020-11-14 06:00:00,2020-11-14 12:00:00,2020-11-14 18:00:00,2020-11-15 00:00:00,2020-11-15 06:00:00,2020-11-15 12:00:00,2020-11-15 18:00:00,2020-11-16 00:00:00,2020-11-16 06:00:00,2020-11-16 12:00:00,2020-11-16 18:00:00,2020-11-17 00:00:00,2020-11-17 06:00:00,2020-11-17 12:00:00,2020-11-17 18:00:00,2020-11-18 00:00:00,2020-11-18 06:00:00,2020-11-18 12:00:00,2020-11-18 18:00:00,2020-11-19 00:00:00,2020-11-19 06:00:00,2020-11-19 12:00:00,2020-11-19 18:00:00,2020-11-20 00:00:00,2020-11-20 06:00:00,2020-11-20 12:00:00,2020-11-20 18:00:00,2020-11-21 00:00:00,2020-11-21 06:00:00,2020-11-21 12:00:00,2020-11-21 18:00:00,2020-11-22 00:00:00,2020-11-22 06:00:00,2020-11-22 12:00:00,2020-11-22 18:00:00,2020-11-23 00:00:00,2020-11-23 06:00:00,2020-11-23 12:00:00,2020-11-23 18:00:00,2020-11-24 00:00:00,2020-11-24 06:00:00,2020-11-24 12:00:00,2020-11-24 18:00:00

In [45]:
X_train = trainset3.iloc[:,:-1]
y_train = trainset3.iloc[:,-1]

In [46]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

In [47]:
param_test ={'min_samples_leaf': sp_randint(6, 100), 
             'min_samples_split': sp_randint(1, 500), 
             'max_depth': sp_randint(4,10),
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'max_features': sp_randint(1, 500)}


In [48]:
clf = GradientBoostingClassifier(random_state=42, n_estimators=30, learning_rate=0.3)
rs = RandomizedSearchCV(estimator=clf, param_distributions=param_test, n_iter=500, scoring='roc_auc', cv=5, refit=True, random_state=42, verbose=True, n_jobs= -1)
rs.fit(X_train, y_train)

Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed: 11.5min
[Parallel(n_jobs=-1)]: Done 1768 tasks      | elapsed: 16.2min
[Parallel(n_jobs=-1)]: Done 2418 tasks      | elapsed: 22.3min
[Parallel(n_jobs=-1)]: Done 2500 out of 2500 | elapsed: 23.2min finished


RandomizedSearchCV(cv=5,
                   estimator=GradientBoostingClassifier(learning_rate=0.3,
                                                        n_estimators=30,
                                                        random_state=42),
                   n_iter=500, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000026DF4845580>,
                                        'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000026C038ACF70>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000026C01C58760>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000026C01C73D30>,
                                        'subsample': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000026C01C73E20>},
                   random_state=42, scoring='ro

In [49]:
print('Best score reached: {} with params: {} '.format(rs.best_score_, rs.best_params_))

Best score reached: 0.83040135 with params: {'max_depth': 4, 'max_features': 392, 'min_samples_leaf': 40, 'min_samples_split': 419, 'subsample': 0.8170162773487566} 


In [61]:
print('Best score reached: {} with params: {} '.format(rs.best_score_, rs.best_params_))

Best score reached: nan with params: {'max_depth': 7, 'max_features': 349, 'min_samples_leaf': 20, 'min_samples_split': 107, 'subsample': 0.8237528002182155} 


In [54]:
# i = rs.best_score_
# best_pa = rs.best_params_
# ini_sc = 0.4
while i < 100:
    i += 1
    clf = GradientBoostingClassifier(random_state=42, n_estimators=30, learning_rate=0.3)
    rs = RandomizedSearchCV(estimator=clf, param_distributions=param_test, n_iter=500, scoring='roc_auc', cv=5, refit=True, random_state=42, verbose=True, n_jobs= 14)
    rs.fit(X_train, y_train)
    if rs.best_score_ > ini_sc:
        ini_sc = rs.best_score_
        best_pa = rs.best_params_
        print(rs.best_score_)
        print(rs.best_params_)

[Parallel(n_jobs=14)]: Using backend LokyBackend with 14 concurrent workers.


Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=14)]: Done  22 tasks      | elapsed:   17.5s
[Parallel(n_jobs=14)]: Done 172 tasks      | elapsed:  2.0min
[Parallel(n_jobs=14)]: Done 422 tasks      | elapsed:  4.8min
[Parallel(n_jobs=14)]: Done 772 tasks      | elapsed:  8.2min
[Parallel(n_jobs=14)]: Done 1222 tasks      | elapsed: 12.6min
[Parallel(n_jobs=14)]: Done 1772 tasks      | elapsed: 17.9min
[Parallel(n_jobs=14)]: Done 2422 tasks      | elapsed: 24.7min
[Parallel(n_jobs=14)]: Done 2500 out of 2500 | elapsed: 25.5min finished


Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=14)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=14)]: Done  22 tasks      | elapsed:   15.6s
[Parallel(n_jobs=14)]: Done 172 tasks      | elapsed:  2.0min
[Parallel(n_jobs=14)]: Done 422 tasks      | elapsed:  4.8min
[Parallel(n_jobs=14)]: Done 772 tasks      | elapsed:  8.3min
[Parallel(n_jobs=14)]: Done 1222 tasks      | elapsed: 12.7min
[Parallel(n_jobs=14)]: Done 1772 tasks      | elapsed: 18.0min
[Parallel(n_jobs=14)]: Done 2422 tasks      | elapsed: 24.5min
[Parallel(n_jobs=14)]: Done 2500 out of 2500 | elapsed: 25.4min finished


Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=14)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=14)]: Done  22 tasks      | elapsed:   15.1s
[Parallel(n_jobs=14)]: Done 172 tasks      | elapsed:  1.9min
[Parallel(n_jobs=14)]: Done 422 tasks      | elapsed:  4.7min
[Parallel(n_jobs=14)]: Done 772 tasks      | elapsed:  8.1min
[Parallel(n_jobs=14)]: Done 1222 tasks      | elapsed: 12.4min
[Parallel(n_jobs=14)]: Done 1772 tasks      | elapsed: 17.7min
[Parallel(n_jobs=14)]: Done 2422 tasks      | elapsed: 24.3min
[Parallel(n_jobs=14)]: Done 2500 out of 2500 | elapsed: 25.1min finished


Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=14)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=14)]: Done  22 tasks      | elapsed:   15.4s
[Parallel(n_jobs=14)]: Done 172 tasks      | elapsed:  2.0min
[Parallel(n_jobs=14)]: Done 422 tasks      | elapsed:  4.8min
[Parallel(n_jobs=14)]: Done 772 tasks      | elapsed:  8.4min


KeyboardInterrupt: 

In [51]:
rs.best_params_

{'max_depth': 4,
 'max_features': 392,
 'min_samples_leaf': 40,
 'min_samples_split': 419,
 'subsample': 0.8170162773487566}

In [52]:
rs.best_score_

0.83040135